In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from datetime import datetime

In [87]:
seed = 42
cuda = True

batch_size = 64
log_interval = 1000

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [88]:
class Net1(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [89]:
class Lenet(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()
        self.conv2_n = 50
        self.hiddens = 50*16
        
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, self.conv2_n, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(self.hiddens, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), 2))
        x = F.tanh(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, self.hiddens)
        x = F.tanh(self.fc1(x))
        x = F.dropout(x, training=self.training, p=0.7)
        x = self.fc2(x)
        return F.log_softmax(x)

In [102]:
model = Lenet()
if cuda:
    model.cuda()

epochs = 40
lr = 0.01
momentum = 0.9

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [103]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [104]:
%%time

start = datetime.now()
new = start

for epoch in range(1, epochs + 1):
    train(epoch)
    test()
    old = new
    new = datetime.now()
    print(">> %s - %s" % (str(new-old), str(new-start)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308010
Test set: Average loss: 0.0831, Accuracy: 9749/10000 (97%)
>> 0:00:09.041813 - 0:00:09.041813
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.058356
Test set: Average loss: 0.0578, Accuracy: 9815/10000 (98%)
>> 0:00:08.023108 - 0:00:17.064921
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.069911
Test set: Average loss: 0.0430, Accuracy: 9868/10000 (99%)
>> 0:00:08.097543 - 0:00:25.162464
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.083620
Test set: Average loss: 0.0457, Accuracy: 9869/10000 (99%)
>> 0:00:07.844977 - 0:00:33.007441
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.042677
Test set: Average loss: 0.0381, Accuracy: 9880/10000 (99%)
>> 0:00:07.945903 - 0:00:40.953344
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.050118
Test set: Average loss: 0.0395, Accuracy: 9887/10000 (99%)
>> 0:00:08.019454 - 0:00:48.972798
Train Epoch: 7 [0/60000 (0%)]	Loss: 0.057563
Test set: Average loss: 0.0401, Accuracy: 9883/10000 (99%)
>> 0:00:07.727856 - 0:00:56.700654
Train Epoch: 8 [0/60000 (0%

In [105]:
model

Lenet (
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d (p=0.5)
  (fc1): Linear (800 -> 512)
  (fc2): Linear (512 -> 10)
)